In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
warnings.filterwarnings('ignore')

In [ ]:
path = '../dsets/liverD/'
train = pd.read_csv(path+'train.csv', index_col='id')
og = pd.read_csv(path+'cirrhosis.csv', index_col='ID')

In [ ]:
test = pd.read_csv(path+'test.csv', index_col='id')

In [ ]:
og.dropna(axis=0, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer

def get_cols(df):
    num_cols = []
    cat_cols = []

    for col in df.columns:
        if df[col].dtypes in ['int64', 'float']:
            num_cols.append(col)
        else:
            cat_cols.append(col)
    
    return num_cols, cat_cols

def normalize(df, cols:list):
    qt = StandardScaler()
    print(df.head(5))
    new_data = qt.fit_transform(df[cols])
    new_data = pd.DataFrame(new_data)
    new_data.columns = df[cols].columns
    df[cols] = new_data[cols]
    print(df[cols].head(5))

def hist_plot(df, df_1, df_2=None, cols=None, cols_1=None, cols_2=None):
    print(cols)
    cols_n0 =len(cols)
    cols_n1 = len(cols_1)
    cols_n2 = len(cols_2) if cols_2 is not None else 0
    print(cols_n0, cols_n1)
    cols_n = cols_n0+cols_n1+cols_n2
    c = 6 if cols_n2 != 0 else 2
    r = math.floor(cols_n/c)
    r = 1 if r == 0 else r
    print(cols_n, r, c)
    plt.subplots(nrows=r, ncols=c, figsize=(20, cols_n), sharey='row')
    plt.subplots_adjust(hspace=0.5)
    #plot.set_xticklabels(plot.get_xticklabels(), rotation=45)

    i = 0

    for col in cols:
        plt.subplot(r, c, i+1)
        plot = sns.histplot(df, x=col)
        plot.set_title('Train')
        i+=1
        plt.subplot(r, c, i+1)
        plot2 = sns.histplot(df_1, x=col)
        plot2.set_title('OG')
        i+=1
        plt.subplot(r, c, i+1)
        plot3 = sns.histplot(df_2, x=col)
        plot3.set_title('Test')
        i+=1
        
def count_plot(df, df_1, df_2=None, cols=None, cols_1=None, cols_2=None):
    cols_n0 =len(cols)
    cols_n1 = len(cols_1) 
    cols_n2 = len(cols_2) if cols_2 is not None else 0
    print(cols_n0, cols_n1)
    cols_n = cols_n0+cols_n1+cols_n2
    c = 6 if cols_n2 != 0 else 2
    r = math.floor(cols_n/c)
    r = 1 if r == 0 else r
    print(cols_n, r, c)
    plt.subplots(nrows=r, ncols=c, figsize=(17, cols_n), sharey='row')
    plt.subplots_adjust(hspace=0.5)
    #plot.set_xticklabels(plot.get_xticklabels(), rotation=45)
    
    i = 0
    
    for col in cols:
        if col == 'Status' and df_2 is not None:
            continue
        plt.subplot(r, c, i+1)
        plot = sns.countplot(df, x=col)
        plot.set_title('Train')
        i+=1
        plt.subplot(r, c, i+1)
        plot2 = sns.countplot(df_1, x=col)
        plot2.set_title('OG')
        i+=1
        if df_2 is None:
            continue
        plt.subplot(r, c, i+1)
        plot3 = sns.countplot(df_2, x=col)
        plot3.set_title('Test')
        i+=1

In [ ]:
num_train, cat_train = get_cols(train)
num_og, cat_og = get_cols(og)
num_test, cat_test = get_cols(test)

In [ ]:
test

In [ ]:
to_one_hot = ['Sex','Drug', 'Edema']
to_label = ['Spiders', 'Hepatomegaly', 'Ascites']

In [ ]:
from sklearn.preprocessing import LabelEncoder

def one_hot(df, cols):
    # label_encoder(df, cols)
    df = pd.get_dummies(df, columns=cols)
    return df

def label_encoder(df, cols):
    le = LabelEncoder()
    for col in cols:
        df[col] = le.fit_transform(df[col])

In [ ]:
#count_plot(train, og, test, cat_train, cat_og, cat_test)

In [ ]:
#count_plot(train, og, cols=['Status'], cols_1=['Status'])

In [ ]:
#hist_plot(train, og, test, num_train, num_og, num_test)

In [ ]:
X = train.drop(['Status'], axis=1)
y = train['Status']
y = y.map({'D':0, 'C':1, 'CL':2})

X_eval = og.drop('Status', axis=1)
y_eval = og['Status']
y_eval = y_eval.map({'D':0, 'C':1, 'CL':2})

In [ ]:
qt = StandardScaler()
fitted_data = qt.fit(X[num_train])
print(fitted_data)
new_data = qt.transform(X[num_train])
new_data = pd.DataFrame(new_data)
new_data.columns = X[num_train].columns
X[num_train] = new_data[num_train]

# normalize(X, num_train)
# normalize(X_eval, num_train)
# normalize(test, num_test)

In [ ]:
new_data = qt.transform(X_eval[num_train])
new_data = pd.DataFrame(new_data)
new_data.columns = X_eval[num_train].columns
X_eval[num_train] = new_data[num_train]

In [ ]:
test = test.reset_index()
new_data = qt.transform(test[num_train])
new_data = pd.DataFrame(new_data)
new_data.columns = test[num_train].columns
test[num_train] = new_data[num_train]

In [ ]:
test.index = test.id
test.drop('id', inplace=True, axis=1)

In [ ]:
#hist_plot(X, X_eval, test, num_train, num_og, num_test)

In [ ]:
# X = one_hot(X, to_one_hot)
# X_eval = one_hot(X_eval, to_one_hot)
# test = one_hot(test, to_one_hot)

In [ ]:
label_encoder(X, to_label)
label_encoder(X_eval, to_label)
label_encoder(test, to_label)

In [ ]:
corr = X[num_train].corr()
plt.figure(figsize=(16, 16))
sns.heatmap(corr, annot=True, fmt='.2f', linewidths=.5, annot_kws={"size": 10})

In [ ]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

xgb = XGBClassifier(objective='multi:softmax', enable_categorical=True, learning_rate=0.01)
ctb = CatBoostClassifier()
knc = KNeighborsClassifier()

In [ ]:
# from sklearn.metrics import log_loss
# 
# his = xgb.fit(X, y)
# y_xgb = xgb.predict_proba(X_eval)
# 
# y_eval_ohe = pd.get_dummies(y_eval)
# loss = log_loss(y_eval_ohe, y_xgb)
# 
# y_pred = xgb.predict_proba(test)
# y_pred

In [ ]:
# sample_submission = pd.read_csv(path + 'sample_submission.csv', index_col='id')
# sample_submission

In [ ]:
# submission = sample_submission.copy()
# submission['Status_D'] = y_pred[:, [0]]
# submission['Status_C'] = y_pred[:, [1]]
# submission['Status_CL'] = y_pred[:, [2]]
# submission.to_csv('./sub1.csv')

In [ ]:
#clipping

def data_preprocess(dataset=None, if_test=None):
    df = dataset.copy()
    df["Drug"] = df["Drug"].map({"Placebo": 0, "D-penicillamine": 1})
    df["Sex"] = df["Sex"].map({"M": 0, "F": 1})
    df['Edema'] = df['Edema'].map({"N":0, "Y":1, "S":2})

    #df = pd.get_dummies(df, columns=['Edema'], drop_first=True)

    for col in ["Ascites", "Hepatomegaly", "Spiders"]:
        df[col] = df[col].map({"N": 0, "Y": 1})
    if not if_test:
        df['Status'] = df['Status'].map({"D": 0, "CL": 1, "C": 2})
    return df

def features_engineering(dataset=None):
    df = dataset.copy()
    df['date_of_diagnosis'] = df['Age'] - df['N_Days']
    df['diseases'] = df['Ascites'] + df['Hepatomegaly'] + df['Spiders'] + df['Edema']
    df['Albumin_Level'] = np.where(df['Albumin'] < 3.4, 1, np.where(df['Albumin'] > 5.4, 2, 0))
    df['Copper_Risk'] = np.where(df['Copper'] > 140, 1, 0)
    df['Platelets_Normal'] = np.where((df['Platelets'] >= 150_000) & (df['Platelets'] <= 450_000),0,1)

    return df

In [ ]:
train = data_preprocess(train, False)
val = data_preprocess(og, False)
test = data_preprocess(test, True)

In [ ]:
train = features_engineering(train)
val = features_engineering(val)
test = features_engineering(test)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

baseline_models = {
    'LGBM': LGBMClassifier(),
    'Logistic Regression': LogisticRegression(multi_class = 'ovr'),
    'XGBoost': XGBClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Hist': HistGradientBoostingClassifier(),
    'Catboost': CatBoostClassifier(loss_function = 'MultiClass')
}

loss_results = pd.DataFrame(columns=['Model', 'Loss'])

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

def evaluation(skf=None,model=None,scoring=None,data=None,target=None,model_name=None):
    X = data.drop(target,axis=1)
    Y = data[target]

    global loss_results
    score = cross_val_score(model,X, Y, scoring = scoring,cv = skf, n_jobs = 10, )
    cv_score = score.mean()*-1
    new_row = {'Model': model_name, 'Loss': cv_score}
    loss_results = pd.concat([loss_results, pd.DataFrame([new_row])], ignore_index=True)

    print(model_name)
    print(f"Loss:{cv_score:.4f}")
    print('====================================')

def plot_evaluation(dataset=None):
    plt.figure(figsize = (10,6))
    bars = plt.barh(y=dataset['Model'],width=dataset['Loss'],color='#82B0D2')
    plt.xlabel('Log_loss')
    plt.title('Model Performance based on Log Loss')
    plt.gca().invert_yaxis()
    for bar in bars:
        yval = bar.get_y() + bar.get_height() / 2
        xval = bar.get_width()
        plt.text(xval, yval, round(xval, 4), va='center', ha='left', color='black')

    plt.show()

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

target = 'Status'
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

for model_name, base_model in baseline_models.items():
    evaluation(skf=rskf, model=base_model,scoring='neg_log_loss',data=train,target=target,model_name=model_name)

In [ ]:
train_total = pd.concat([train, val], axis='rows', ignore_index=True).reset_index()
train_total.drop_duplicates(inplace=True)
train_total.drop('index', axis=1, inplace=True)

In [ ]:
y = train_total['Status']
X = train_total.drop('Status', axis=1)

In [ ]:
#evaluation(skf=rskf, model=CatBoostClassifier(loss_function='MultiClass'),scoring='neg_log_loss',data=train_total,target=target,model_name='Catboost+FE+Origin Data')

In [ ]:
from sklearn.metrics import log_loss

n_splits = 5
n_repeats = 3

In [ ]:
from catboost import CatBoostClassifier

patience = 100
model_cb = []
pred_cb = []

k_fold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

cbr_param = {
    'iterations': 470,
    'depth': 16,
    'learning_rate': 0.1,
    'l2_leaf_reg': 4,
    'max_bin': 238,
    'od_wait': 49,
    'one_hot_max_size': 39,
    'min_data_in_leaf': 11,
    'grow_policy': 'Lossguide',
    'bootstrap_type': 'Bernoulli',
    'task_type': 'GPU',
    'devices': '0'
}

for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_valid, y_valid = X.iloc[test_index], y.iloc[test_index]
    
    model_c = CatBoostClassifier(**cbr_param)
    
    model_c.fit(X=X_train, y=y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = patience)
    
    model_cb.append(model_c)
    pred_cb.append(model_c.predict_proba(test))
    print(f'index: {i}')

In [ ]:
from xgboost import XGBClassifier

k_fold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

modelsXB = []
predsXB = []

xgbr_params ={
    'objective': 'multi_logloss',
    'device': 'gpu',
    'tree_method':'gpu_hist',
    'max_depth': 9, 'min_child_weight': 8,
    'learning_rate': 0.0337716365315986,
    'n_estimators': 733,
    'subsample': 0.6927955384688348,
    'colsample_bytree': 0.1234702658812108,
    'reg_alpha': 0.18561628377665318,
    'reg_lambda': 0.5565488299127089,
    'random_state': 42
}


for train_index, test_index in k_fold.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

    modelx = XGBClassifier(**xgbr_params)

    modelx.fit(X=X_train, y=y_train,
               eval_set=[(X_valid, y_valid)],
               early_stopping_rounds = patience,
               verbose = 100
               )
    modelsXB.append(modelx)
    predsXB.append(modelx.predict_proba(test))

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

k_fold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

modelsHGBC = []
predsHGBC = []

PATIENCE = 50

hgbc_params = {
    'l2_regularization': 5,
    'learning_rate': 0.065,
    'max_iter': 300,
    'max_depth': 15,
    'max_bins': 250,
    'min_samples_leaf': 10,
    'max_leaf_nodes': 5,
    'random_state': 3,
}
i = 0
for train_index, test_index in k_fold.split(X, y):
    i += 1
    print('repeat: ', i)
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

    modelhgbc = HistGradientBoostingClassifier(**hgbc_params)

    modelhgbc.fit(X_train, y_train)
    modelsHGBC.append(modelhgbc)
    predsHGBC.append(modelhgbc.predict_proba(test))

In [ ]:
from lightgbm import LGBMClassifier

k_fold = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)
modelsLB = []
pred_lb = []
patience = 50

lgbr_param = {
    'device_type': 'gpu',
    'early_stopping_round': 50,
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.5,
    'importance_type': 'gain',
    'n_estimators': 800,
    'learning_rate': 0.1,
    'random_state': 27,
    'reg_alpha': 0,
    'reg_lambda': 248,
    'subsample': 0.8768432946977912
}

i = 0
for train_index, test_index in k_fold.split(X, y):
    i += 1
    print('repeat: ', i)
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]

    modelgb = LGBMClassifier(**lgbr_param)

    modelgb.fit(X=X_train, y=y_train,
                eval_set=[(X_valid, y_valid)],
                )
    modelsLB.append(modelgb)
    pred_lb.append(modelgb.predict_proba(test))

In [ ]:
df_sub = pd.DataFrame({'Status_C':[], 'Status_CL':[], 'Status_D':[]})

In [ ]:
df_sub['Status_C'] = (modelhgbc.predict_proba(test)[:, 0])*0.15 + (modelx.predict_proba(test)[:, 0])*0.45 + (modelgb.predict_proba(test)[:, 0]*0.4)+(model_c.predict_proba(test)[:, 0]*0.001)
df_sub['Status_D'] = (modelhgbc.predict_proba(test)[:, 2])*0.15 + (modelx.predict_proba(test)[:, 2])*0.45 + (modelgb.predict_proba(test)[:, 2]*0.4)+(model_c.predict_proba(test)[:, 2]*0.001)
df_sub['Status_CL'] = (modelhgbc.predict_proba(test)[:, 1])*0.15 + (modelx.predict_proba(test)[:, 1])*0.45 + (modelgb.predict_proba(test)[:, 1]*0.4)+(model_c.predict_proba(test)[:, 1]*0.001)

df_sub.index = pd.read_csv(path+'sample_submission.csv')['id']
df_sub
# df_sub.to_csv('./sub2.csv')

In [ ]:
base_lgb = LGBMClassifier()
base_lgb.fit(X,y)
base_lgb.get_params()

In [ ]:
plot = sns.barplot(x=base_lgb.feature_name_, y=base_lgb.feature_importances_)
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)

In [ ]:
X.corr()

In [ ]:
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from optuna import trial

In [ ]:
def objective_lgb(trial, data=X,target=y):
    X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.2)

    params = {'boosting_type': trial.suggest_categorical('boosting_type',['gbdt','dart']),
              'colsample_bytree': trial.suggest_categorical('colsample_bytree',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
              'importance_type': trial.suggest_categorical('importance_type',['split','gain']),
              'n_estimators': trial.suggest_categorical('n_estimators',[10,50,100,250,300,350,400,500,800,1000]),
              'learning_rate': trial.suggest_categorical('learning_rate',[0.01,0.02,0.05,0.1,]),
              'random_state': trial.suggest_int('random_state',1,45),
              'reg_alpha': trial.suggest_int('reg_alpha',0,1000),
              'reg_lambda': trial.suggest_int('reg_lambda',0,1000),
              'subsample': trial.suggest_float('subsample',0.8,1.0),
              'min_child_samples': 20,
              'min_child_weight': 0.001,
              'num_leaves': 31,
              'subsample_for_bin': 200000,
              'early_stopping_round': 50,
              'subsample_freq': 0,
              'max_depth': -1,
              'device_type': 'gpu'
              }
    
    modelgb = LGBMClassifier(objective='multiclass', metrics='multi_logloss', **params)
    modelgb.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    pred = modelgb.predict_proba(X_val)
    score = log_loss(y_val, pred, labels=[0, 1, 2])
    return score
    

In [ ]:
study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=100)

In [ ]:
best_params = study_lgb.best_params
best_params['device_type'] = 'GPU'

In [ ]:
best_params

In [ ]:
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

optuna_lgb = LGBMClassifier(**study_lgb.best_params)
rskf = RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=42)
evaluation(skf=rskf, model=optuna_lgb, scoring='neg_log_loss', data=train_total, target='Status', model_name='Optuna LGBM')

In [ ]:
optuna_lgb.fit(X, y)

In [ ]:
pred_optuna_lgb = optuna_lgb.predict_proba(test)

In [ ]:
pred_optuna_lgb

In [ ]:
pred_optuna_lgb = pd.DataFrame(pred_optuna_lgb, columns=['D', 'CL', 'C'], index=pd.read_csv(path+'sample_submission.csv')['id'])

In [ ]:
pred_optuna_lgb

In [ ]:
X

In [ ]:
train_corr = train.corr()

In [ ]:
sns.histplot(train, y='Status')

In [ ]:
reverse_dummy = pd.from_dummies(pred_optuna_lgb)
sns.histplot(pred_optuna_lgb)

In [ ]:
pred_optuna_lgb.rename(columns={'D':0, 'CL':1, 'C':2})
pred_optuna_lgb = pred_optuna_lgb.apply(lambda x: x.argmax(0))

In [ ]:
pred_optuna_lgb